# Оценка качества поиска

Структура датасета: 

id	query_id	total_result	visited_or_related_document_id	order_number_of_the_document	assessment_of_the_document	judgement_of_the_document

In [6]:
!pip install faiss-cpu annoy

In [7]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
from annoy import AnnoyIndex

In [8]:
metadata = pd.read_csv('metadata.csv')
metadata.head(5)

,Unnamed: 0,id,product_title,product_description,filename,format,mode,resolution,filesize,retrieved,file_exists,merged_text
0,0,209650955,Триммер садовый электрический для травы,Модель беспроводной; аккумуляторный; ручной Ти...,res/img/209650955.png,PNG,RGBA,"(246, 328)",171.35,2024-03-21 11:10:04.299254016,True,триммер садовый электрический для травы модель...
1,1,77278318,Триммер аккумуляторный садовый дачный кусторез,Модель разборная модель Питание Аккумулятор (в...,res/img/77278318.png,PNG,RGBA,"(246, 328)",160.00,2024-03-21 11:13:44.241293056,True,триммер аккумуляторный садовый дачный кусторез...
2,2,18565737,Триммер аккумуляторный ZITREK GreenCut 20 (20В...,Модель GreenCut 20 Гарантийный срок 1 год Тип ...,res/img/18565737.png,PNG,RGBA,"(246, 328)",17.67,2024-03-21 11:13:58.767094016,True,триммер аккумуляторный zitrek greencut 20 (20в...
3,3,113066515,Триммер аккумуляторный садовый дачный кусторез,Модель разборная модель Питание Аккумулятор (в...,res/img/113066515.png,PNG,RGBA,"(246, 328)",128.90,2024-03-21 11:14:13.620366080,True,триммер аккумуляторный садовый дачный кусторез...
4,4,151530782,Триммер-газонокосилка аккумуляторный ZITREK Gr...,Модель GreenCut 20 Pro Гарантийный срок 1 год ...,res/img/151530782.png,PNG,RGBA,"(246, 328)",128.13,2024-03-21 11:14:27.381707008,True,триммергазонокосилка аккумуляторный zitrek gre...


In [9]:
# Sample documents
documents = metadata[['id', 'product_title', 'merged_text', 'filename']]

# Convert text data into TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents['merged_text'])
vectors = tfidf_matrix.toarray()

### Инициализация [FAISS](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes) и [Annoy]() с разными методами 

In [ ]:
# Indexing with Faiss
d = vectors.shape[1]

# FAISS L2
faiss_L2 = faiss.IndexFlatL2(d)  
faiss_L2.add(vectors.astype(np.float32))

# FAISS Inner Product
faiss_IP = faiss.IndexFlatIP(d)  # Inner product  
faiss_IP.add(vectors.astype(np.float32))

# Annoy Angular
num_trees = 10
annoy_angular = AnnoyIndex(d, 'angular')
for i, vector in enumerate(vectors):
    annoy_angular.add_item(i, vector)
annoy_angular.build(num_trees)

# Annoy Euclidean
num_trees = 10
annoy_angular = AnnoyIndex(d, 'euclidean')
for i, vector in enumerate(vectors):
    annoy_angular.add_item(i, vector)
annoy_angular.build(num_trees)

